In [ ]:
# === 1. KÖRNYEZET BEÁLLÍTÁSA ===
# Könyvtárak telepítése és importálása
%pip install -U torch sentence-transformers accelerate pyarrow pandas tqdm transformers

import pandas as pd
import numpy as np
import gc
import json
import pyarrow.parquet as pq
from sentence_transformers import SentenceTransformer
import torch
import time
from tqdm.auto import tqdm
from typing import List
from pathlib import Path
import os

# A PyTorch rugalmasabban kezelje a GPU memóriát.
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# GPU optimalizáció A100-hoz
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

print(f"CUDA elérhető: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# === 2. KONFIGURÁCIÓ ===
# RunPod A100 felhő környezethez igazított konfiguráció.

from pathlib import Path

# Bemeneti és kimeneti fájlok
# Győződj meg róla, hogy a fájlok a megfelelő helyen vannak a felhő környezetben!
INPUT_CSV_PATH = Path("/workspace/cleaned_data_for_embedding.csv")
OUTPUT_PARQUET_PATH = Path("/workspace/documents_with_embeddings.parquet")

# Modell és batch méret beállítások
MODEL_NAME = "Qwen/Qwen3-Embedding-0.6B"
EMBEDDING_DIMENSION = 1024
# A100 kártyán egy nagyobb batch méret is hatékony lehet
BATCH_SIZE = 256

print(f"Input: {INPUT_CSV_PATH}")
print(f"Output: {OUTPUT_PARQUET_PATH}")
print(f"Modell: {MODEL_NAME}")
print(f"Batch méret: {BATCH_SIZE}")

In [ ]:
# === 3. EMBEDDING GENERÁTOR OSZTÁLY ===
class EmbeddingGenerator:
    def __init__(self, model_name: str, batch_size: int, dimension: int, device: str = 'cuda'):
        self.model_name = model_name
        self.batch_size = batch_size
        self.dimension = dimension
        self.device = device if torch.cuda.is_available() else 'cpu'
        self.model = None
        print(f"Generátor inicializálva a(z) '{self.device}' eszközön.")

    def load_model(self):
        if self.model is not None:
            print("Modell már be van töltve.")
            return
        try:
            print(f"'{self.model_name}' modell betöltése...")
            self.model = SentenceTransformer(self.model_name, device=self.device, trust_remote_code=True)
            self._warmup_model()
            print("Modell sikeresen betöltve és bemelegítve.")
        except Exception as e:
            print(f"Modell betöltési hiba: {e}")
            raise

    def _warmup_model(self):
        print("Modell bemelegítése...")
        # Egy rövid szöveggel "bemelegítjük" a modellt
        self.generate_embeddings(["melegítés"])
        self._cleanup_memory()
        print("Bemelegítés kész.")

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        if self.model is None:
            raise RuntimeError("A modell nincs betöltve. Hívd meg a load_model() metódust.")
        
        embeddings = self.model.encode(
            texts, 
            batch_size=self.batch_size, 
            normalize_embeddings=True, 
            show_progress_bar=True,
            convert_to_numpy=True
        )
        
        # Biztonsági ellenőrzés a dimenzióra
        if embeddings.shape[1] != self.dimension:
            print(f"Figyelmeztetés: Váratlan embedding dimenzió: {embeddings.shape[1]}. Korrekció {self.dimension}-ra.")
            embeddings = embeddings[:, :self.dimension]
            
        return embeddings.astype(np.float32)

    def _cleanup_memory(self):
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

In [ ]:
# === 4. FŐ FELDOLGOZÁSI FOLYAMAT ===
def create_metadata_json(row: pd.Series) -> str:
    """Létrehoz egy JSON stringet a sor metaadataiból."""
    metadata_cols = [col for col in row.index if col not in ['text', 'embedding']]
    metadata_dict = row[metadata_cols].dropna().to_dict()
    return json.dumps({k: str(v) for k, v in metadata_dict.items()}, ensure_ascii=False)

def main():
    print("Feldolgozás indítása...")
    start_time = time.time()

    # Bemeneti adatok beolvasása
    if not INPUT_CSV_PATH.exists():
        raise FileNotFoundError(f"Hiba: A bemeneti fájl nem található: {INPUT_CSV_PATH}")
    
    print(f"Bemeneti CSV beolvasása: {INPUT_CSV_PATH}")
    # Egyszerűsített beolvasás, a C motor használatával
    df = pd.read_csv(INPUT_CSV_PATH)
    print(f"{len(df):,} sor sikeresen beolvasva.")

    # Szövegek kinyerése
    df['text'] = df['text'].fillna('')
    texts_to_process = df['text'].astype(str).tolist()
    
    if not texts_to_process:
        print("Nincs feldolgozandó szöveg a bemeneti fájlban.")
        return

    # Embedding generátor inicializálása és modell betöltése
    generator = EmbeddingGenerator(MODEL_NAME, BATCH_SIZE, EMBEDDING_DIMENSION)
    generator.load_model()

    # Embedding generálás a teljes adathalmazon
    print("Embedding generálás megkezdése...")
    embeddings = generator.generate_embeddings(texts_to_process)
    
    # Memória takarítás a nagy művelet után
    generator._cleanup_memory()

    # Eredmények hozzáadása a DataFrame-hez
    if len(embeddings) == len(df):
        df['embedding'] = list(embeddings)
    else:
        print(f"KRITIKUS HIBA: Az embeddingek száma ({len(embeddings)}) nem egyezik a DataFrame sorainak számával ({len(df)}). A program leáll.")
        return

    # Metaadatok generálása
    print("Metaadat JSON generálása...")
    df['metadata_json'] = [create_metadata_json(row) for _, row in tqdm(df.iterrows(), total=len(df), desc="Metaadat JSON")]

    # Kimeneti DataFrame és mentés Parquet formátumba
    final_df = df[['doc_id', 'text', 'embedding', 'metadata_json']]
    OUTPUT_PARQUET_PATH.parent.mkdir(parents=True, exist_ok=True)
    print(f"Eredmények mentése a Parquet fájlba: {OUTPUT_PARQUET_PATH}")
    final_df.to_parquet(OUTPUT_PARQUET_PATH, index=False, compression='snappy')
    
    # Összegzés
    total_rows_processed = len(final_df)
    total_time_seconds = time.time() - start_time
    rows_per_second = total_rows_processed / total_time_seconds if total_time_seconds > 0 else 0
    
    print("\n" + "="*50)
    print("✅ FELDOLGOZÁS BEFEJEZVE")
    print(f"📄 Kimeneti fájl: {OUTPUT_PARQUET_PATH}")
    print(f"⏱️ Teljes idő: {total_time_seconds:.2f} másodperc ({total_time_seconds / 60:.2f} perc)")
    print(f"📊 Feldolgozott sorok: {total_rows_processed:,}")
    print(f"⚡ Átlagos sebesség: {rows_per_second:.2f} sor/mp")
    print("="*50)

# Fő folyamat futtatása
main()

In [ ]:
# === 5. VALIDÁCIÓ ===
print("Kimeneti Parquet fájl validálása...")

if OUTPUT_PARQUET_PATH.exists():
    try:
        parquet_file = pq.ParquetFile(OUTPUT_PARQUET_PATH)
        file_num_rows = parquet_file.metadata.num_rows
        file_size_mb = OUTPUT_PARQUET_PATH.stat().st_size / (1024 * 1024)
        
        df_sample = pd.read_parquet(OUTPUT_PARQUET_PATH, engine='pyarrow').head(5)
        sample_embedding = df_sample['embedding'].iloc[0]
        
        print("\n✅ VALIDÁCIÓ SIKERES!")
        print(f"  Fájl méret: {file_size_mb:.2f} MB")
        print(f"  Sorok száma: {file_num_rows:,}")
        print(f"  Oszlopok: {df_sample.columns.tolist()}")
        print(f"  Első embedding dimenziója: {len(sample_embedding)}")
        print("\n--- Minta Adatsor ---")
        display(df_sample)
        
    except Exception as e:
        print(f"Hiba a Parquet fájl validálása közben: {e}")
        print(f"\n❌ HIBA a validáció során: {e}")
else:
    print("A kimeneti Parquet fájl nem jött létre.")
    print("\n❌ HIBA: A kimeneti fájl nem található!")